# Getting Started with QuantConnect

## 1. Numpy and pandas

In [23]:
import numpy as np

price_list = [143.73, 145.83, 143.68, 144.02, 143.5, 142.62]
price_array = np.array(price_list)
print(price_array, type(price_array))

(array([143.73, 145.83, 143.68, 144.02, 143.5 , 142.62]), numpy.ndarray)

In [24]:
Ar = np.array([[1,2],[3,4]])
print(Ar, Ar.shape)

(array([[1, 2],
        [3, 4]]), (2, 2))

In [25]:
print("rows:")
print(Ar[0])
print(Ar[1])
print(Ar[:,0])
print(Ar[:,1])

rows:
[1 2]
[3 4]
[1 3]
[2 4]


In [26]:
print(price_array)
print(np.log(price_array))
print(np.mean(price_array))
print(np.std(price_array))

[143.73 145.83 143.68 144.02 143.5  142.62]
[4.96793654 4.98244156 4.9675886  4.96995218 4.96633504 4.96018375]
143.89666666666668
0.9673790478515796


In [29]:
import pandas as pd

s = pd.Series(price_list)
s

0    143.73
1    145.83
2    143.68
3    144.02
4    143.50
5    142.62
dtype: float64

In [30]:
s[:3]

0    143.73
1    145.83
2    143.68
dtype: float64

In [31]:
s[4] = 0
s

0    143.73
1    145.83
2    143.68
3    144.02
4      0.00
5    142.62
dtype: float64

In [33]:
time_index = pd.date_range('2017-01-01', periods=len(s), freq='D')
s = pd.Series(price_list, index=time_index, name='Apple prices')
print(s)
print(s.iloc[0])
print(s['2017-01-03':'2017-01-06'])

2017-01-01    143.73
2017-01-02    145.83
2017-01-03    143.68
2017-01-04    144.02
2017-01-05    143.50
2017-01-06    142.62
Freq: D, Name: Apple prices, dtype: float64
143.73
2017-01-03    143.68
2017-01-04    144.02
2017-01-05    143.50
2017-01-06    142.62
Freq: D, Name: Apple prices, dtype: float64


In [35]:
print(s.describe())
print(s[s < np.mean(s)])
print(s[(s > np.mean(s)) & (s < np.mean(s) + 1.64*np.std(s))])

count      6.000000
mean     143.896667
std        1.059711
min      142.620000
25%      143.545000
50%      143.705000
75%      143.947500
max      145.830000
Name: Apple prices, dtype: float64
2017-01-01    143.73
2017-01-03    143.68
2017-01-05    143.50
2017-01-06    142.62
Name: Apple prices, dtype: float64
2017-01-04    144.02
Freq: D, Name: Apple prices, dtype: float64


## 2. Resampling and Dataframes

In [10]:
import quandl
quandl.ApiConfig.api_key = 'QQ4gF5jjG1U5ssS_h3-M'

aapl_table = quandl.get('WIKI/AAPL') # this is a pandas dataframe (see below)

In [37]:
aapl = aapl_table['Adj. Close']['2017'] # adj. means adjusted for dividends before next day's open
print(aapl['2017-3'].head(10))
print(type(aapl)) #this is a pandas Series object
print(aapl.idxmax())

Date
2017-03-01    138.657681
2017-03-02    137.834404
2017-03-03    138.647762
2017-03-06    138.211326
2017-03-07    138.389868
2017-03-08    137.874080
2017-03-09    137.556672
2017-03-10    138.012946
2017-03-13    138.072460
2017-03-14    137.864161
Name: Adj. Close, dtype: float64

In [55]:
print("Monthly average prices")
aapl_month_avg = aapl.resample('M').mean() #resample splits data into regular intervals
print(aapl_month_avg) #monthly data
print("\n\nEnd of month prices")
aapl_last_day = aapl.resample('M').agg(lambda x: x[-1]) # this returns the price in the last day of each month
print(aapl_last_day.head())
print("\n\nMonthly returns")
aapl_month_returns = aapl.resample('M').agg(lambda x: (x[-1]-x[0])/x[0])
print(aapl_month_returns.head())
print("\naverage return:", aapl_month_returns.mean())

Monthly average prices
Date
2017-01-31    118.093136
2017-02-28    132.456268
2017-03-31    139.478802
2017-04-30    141.728436
2017-05-31    151.386305
2017-06-30    147.233064
2017-07-31    147.706190
2017-08-31    158.856375
2017-09-30    157.606500
2017-10-31    157.811627
2017-11-30    172.214500
2017-12-31    171.893100
Freq: M, Name: Adj. Close, dtype: float64


End of month prices
Date
2017-01-31    119.851150
2017-02-28    135.880362
2017-03-31    142.496334
2017-04-30    142.486415
2017-05-31    152.142689
Freq: M, Name: Adj. Close, dtype: float64


Monthly returns
Date
2017-01-31    0.044770
2017-02-28    0.068580
2017-03-31    0.027684
2017-04-30   -0.000348
2017-05-31    0.046283
Freq: M, Name: Adj. Close, dtype: float64

average return: 0.02661869365844138


In [56]:
print(aapl_last_day.diff())
print(aapl_last_day.pct_change().fillna(0))
#print(aapl_last_day.pct_change().fillna(method="bfill"))
#print(aapl_last_day.pct_change().dropna())

Date
2017-01-31          NaN
2017-02-28    16.029211
2017-03-31     6.615972
2017-04-30    -0.009919
2017-05-31     9.656274
2017-06-30    -8.704681
2017-07-31     4.810482
2017-08-31    15.751511
2017-09-30    -9.880000
2017-10-31    14.920000
2017-11-30     2.810000
2017-12-31    -2.620000
Freq: M, Name: Adj. Close, dtype: float64
Date
2017-01-31    0.000000
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


In [57]:
# a dataframe is a collection of series
dict = {'AAPL': [143.5,  144.09, 142.73, 144.18, 143.77],
        'GOOG': [898.7,  911.71, 906.69, 918.59, 926.99],
        'IBM':  [155.58, 153.67, 152.36, 152.94, 153.49]}
dates = pd.date_range('2017-07-03', periods = 5, freq = 'D')
pd.DataFrame(dict, index = dates)

,AAPL,GOOG,IBM
2017-07-03,143.50,898.70,155.58
2017-07-04,144.09,911.71,153.67
2017-07-05,142.73,906.69,152.36
2017-07-06,144.18,918.59,152.94
2017-07-07,143.77,926.99,153.49


In [60]:
df = aapl_table
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.422706,0.424470,0.422706,0.422706,117258400.0
1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.402563,0.402563,0.400652,0.400652,43971200.0
1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.373010,0.373010,0.371246,0.371246,26432000.0
1980-12-17,25.87,26.00,25.87,25.87,385900.0,0.0,1.0,0.380362,0.382273,0.380362,0.380362,21610400.0
1980-12-18,26.63,26.75,26.63,26.63,327900.0,0.0,1.0,0.391536,0.393300,0.391536,0.391536,18362400.0


In [45]:
df[['Adj. Open', 'Volume']].head()

,Adj. Open,Volume
Date,,
1980-12-12,0.422706,2093900.0
1980-12-15,0.402563,785200.0
1980-12-16,0.373010,472000.0
1980-12-17,0.380362,385900.0
1980-12-18,0.391536,327900.0


In [41]:
# all series methods can be applied to dataframes
aapl_2016_last_day = df['2016'].resample('M').agg(lambda x: x[-1])
aapl_2016_last_day[['Open', 'Close', 'Volume']].head()

,Open,Close,Volume
Date,,,
2016-01-31,94.79,97.34,64416504.0
2016-02-29,96.86,96.69,35216277.0
2016-03-31,109.72,108.99,25888449.0
2016-04-30,93.99,93.74,68531478.0
2016-05-31,99.60,99.86,42307212.0


In [62]:
aapl_2016_last_day['rate_return'] = aapl_2016_last_day.Close.pct_change()
aapl_2016_last_day[['Open','Close','Volume', 'rate_return']].head()

,Open,Close,Volume,rate_return
Date,,,,
2016-01-31,94.79,97.34,64416504.0,NaN
2016-02-29,96.86,96.69,35216277.0,-0.006678
2016-03-31,109.72,108.99,25888449.0,0.127211
2016-04-30,93.99,93.74,68531478.0,-0.139921
2016-05-31,99.60,99.86,42307212.0,0.065287


In [68]:
missing = aapl_2016_last_day.isnull()
missing.describe()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,rate_return
count,12,12,12,12,12,12,12,12,12,12,12,12,12
unique,1,1,1,1,1,1,1,1,1,1,1,1,2
top,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,12,12,12,12,12,12,12,12,12,12,12,12,11


In [69]:
missing[missing.rate_return==True] #this is the row with a NaN value

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,rate_return
Date,,,,,,,,,,,,,
2016-01-31,False,False,False,False,False,False,False,False,False,False,False,False,True


In [72]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1, s2], axis = 1)
data_frame

,AAPL,GOOG
0,143.50,898.70
1,144.09,911.71
2,142.73,906.69
3,144.18,918.59
4,143.77,926.99


In [84]:
s1 = df.Close.resample('M').agg(lambda x: x[-1])['2017-01-01':'2017-12-31']
s2 = df.Volume.resample('M').agg(lambda x: x[-1])['2016-07-01':'2017-07-31']
pd.concat([s1,s2], axis=1)

,Close,Volume
Date,,
2016-07-31,NaN,27733688.0
2016-08-31,NaN,29662406.0
2016-09-30,NaN,36379106.0
2016-10-31,NaN,26419398.0
2016-11-30,NaN,36162258.0
2016-12-31,NaN,30586265.0
2017-01-31,121.35,49200993.0
2017-02-28,136.99,23482860.0
2017-03-31,143.66,19661651.0


In [85]:
pd.concat([s1,s2], axis=1, join='inner')

,Close,Volume
Date,,
2017-01-31,121.35,49200993.0
2017-02-28,136.99,23482860.0
2017-03-31,143.66,19661651.0
2017-04-30,143.65,20247187.0
2017-05-31,152.76,23162873.0
2017-06-30,144.02,22328979.0
2017-07-31,148.85,19422655.0
